In [1]:
import sys
import os
import numpy as np
import multiprocessing
import dill
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


from util_functions import *
from uncertainty_analysis import *
from sim_helper_functions import *


module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path + "/src/simulations_v2")
from load_params import load_params

from multi_group_simulation import MultiGroupSimulation

In [2]:
lockdown_matrix_str = \
"""6.25	2	0.05	0.05	0	0	0	0.02
1.70	4	0.05	0.05	0	0	0	0.04
0.09	0.11	2	0.05	0	0	0	0.36
0.07	0.08	0.04	4.5	0	0	0	0.04
0.00	0.00	0.00	0.00	0.5	0.075	0.15	0.78
0.00	0.00	0.00	0.00	0.14	0.9	0.1	0.78
0.00	0.00	0.00	0.00	0.11	0.04	0.9	0.78
0.002	0.005	0.021	0.003	0.045	0.024	0.060	3.5"""

lockdown_matrix = np.array([[float(entry) for entry in line.split('\t')] for line in lockdown_matrix_str.split('\n')])

In [3]:
lhs_output_sim_files = []
for i in range(2000):
    fname = '/home/jmc678/covid_data/group-testing/notebooks/apr_29_scenarios/point_{}.dill'.format(i)
    lhs_output_sim_files.append(fname)

scenario_data = load_sim_output(lhs_output_sim_files)
res_results = residential_regression(scenario_data)
res_pessimistic = calculate_pessimistic_scenario(res_results)
centre = get_centre_point()


In [6]:
def run_multigroup_sim(sim, T):
    lockdown_statuses, sim_df = sim.run_new_trajectory(12*7)
    popsize = sim.get_total_population()
    infection_counts = sim_df[['S', 'QS']].apply(lambda x: (popsize - sum(x)) / popsize * 100, axis=1)
    inf_list = list()
    hosp_list = list()
    for group in sim.sims:
        df = group.sim_df
        inf_list.append(get_cum_infections(df))
        hosp_list.append(get_cum_hosp(df))
    return inf_list, hosp_list, lockdown_statuses, list(infection_counts)

def run_multiple_trajectories(sim, T, n):
    inf_matrix = list()
    hosp_matrix = list()
    lockdown_trajectories = []
    infection_trajectories = []
    for _ in range(n):
        inf, hosp, lockdowns, infections = run_multigroup_sim(sim, T)
        inf_matrix.append(inf)
        hosp_matrix.append(hosp)
        lockdown_trajectories.append(lockdowns)
        infection_trajectories.append(infections)
    return inf_matrix, hosp_matrix, lockdown_trajectories, infection_trajectories

def evaluate_testing_policy_v2(params_list, interaction_matrix, group_names, test_frac, T, n,
                           post_lockdown_matrix, new_cases_threshold, new_cases_time_window):
    assert len(params_list) == len(test_frac)
    sim = MultiGroupSimulation(params_list, interaction_matrix, group_names)

    sim.configure_lockdown(post_lockdown_matrix, [True] * 7 + [False], new_cases_threshold, new_cases_time_window)
    
    inf_matrix, hosp_matrix, lockdown_trajectories, infection_trajectories = run_multiple_trajectories(sim, T, n)
    
    return lockdown_trajectories, infection_trajectories

def simulate_lockdown_v2(params, infections_thresh, infections_window, nreps=10):
    simulation_population = 90890
    new_cases_thresh = infections_thresh
#     new_cases_thresh = infections_thresh / simulation_population
    test_policy = [2/7,2/7,1/7,1/7,2/7,1/7,1/30,0]
    
    processed_params, _ = uncertainty_point_to_params_dict(params)
    
    params_dict = processed_params[0]
    interaction_matrix = processed_params[1]
    group_names = processed_params[2]
    
    return evaluate_testing_policy_v2(params_dict,
                                      interaction_matrix,
                                      group_names,
                                      test_policy,
                                      112,
                                      nreps,
                                      lockdown_matrix,
                                       new_cases_thresh,
                                       infections_window)

In [18]:
import multiprocessing, time

def run_sim_wrapper(params, infections_thresh, infections_window,
                    proc_key, nreps, random_seed, return_dict):
    np.random.seed(random_seed)
    lockdowns, final_infections = simulate_lockdown_v2(params, 
                                                    infections_thresh, 
                                                    infections_window,
                                                    nreps=nreps)
    return_dict[proc_key] = {'lockdown_pct': lockdowns,
                            'final_infection_pct': final_infections}


BASE_SEED = 1000000
    
def simulate_protocol_multiprocessing(infections_thresh, infections_window, nreps):
    manager = multiprocessing.Manager()
    return_dict = manager.dict()

    jobs = []
    
    base_seed = np.random.randint(1000000, 10000000)

    params_list = get_points_on_line(centre, res_pessimistic, npoints=13, mult_lb=0, mult_ub=1.5)
    for idx, params in enumerate(params_list):
        print(idx+base_seed)
        p = multiprocessing.Process(target = run_sim_wrapper,
                                   args = (params, infections_thresh, infections_window, idx, nreps, 
                                           idx + base_seed, return_dict))
        jobs.append(p)
        p.start()

    for p in jobs:
        p.join()
    
    return return_dict


In [19]:
nreps = 5
return_dicts = {}
infection_thresholds = [150,200,250,300,350]
infection_windows = [7]


import itertools

for thresh, window in itertools.product(infection_thresholds, infection_windows):
    start = time.time()
    return_dict = simulate_protocol_multiprocessing(thresh, window, nreps)
    end = time.time()
    return_dicts[(thresh, window)] = {key:val for key,val in return_dict.items()}
    print("done simulating {} in {:.2f} seconds".format((thresh, window), end-start))


3453534
3453535
3453536
3453537
3453538
3453539
3453540
3453541
3453542
3453543
3453544
3453545
3453546
done simulating (150, 7) in 44.50 seconds
7783291
7783292
7783293
7783294
7783295
7783296
7783297
7783298
7783299
7783300
7783301
7783302
7783303
done simulating (200, 7) in 44.42 seconds
4544965
4544966
4544967
4544968
4544969
4544970
4544971
4544972
4544973
4544974
4544975
4544976
4544977
done simulating (250, 7) in 44.75 seconds
7903666
7903667
7903668
7903669
7903670
7903671
7903672
7903673
7903674
7903675
7903676
7903677
7903678
done simulating (300, 7) in 44.38 seconds
2149431
2149432
2149433
2149434
2149435
2149436
2149437
2149438
2149439
2149440
2149441
2149442
2149443
done simulating (350, 7) in 44.34 seconds


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(10,6))

mults = [1,1.2,1.4,1.6,1.8,2, 2.2, 2.4, 2.6, 2.8, 3]

keys = list(return_dicts.keys())

xlabel_mults = [(mult-1)*100 for mult in mults]

for key in keys:
    if key[0] == 400:
        continue
    return_dict = return_dicts[key]
    infections = []
    for mult in mults:
        infections.append(np.mean([inf[-1] / 100 * popsize for inf in return_dict[mult]['final_infection_pct']]))
    plt.plot(xlabel_mults, infections, label="Shutdown if > {} cases in 1 week".format(key[0]))

plt.xlabel("Percentage Increase to Nominal On-Campus Transmission Rate")
plt.ylabel("Average Total Infections")
plt.title("Total Infections Under Various Shutdown Protocols")
plt.legend(loc='best')

#plt.savefig('res_inst_paper_graphs/mar_19_graphs/shutdown_infections.png', dpi=450)